In [50]:
import os

import sqlite3
import pandas

In [102]:
#connection.commit()
connection.close()
os.unlink('data/l1000.db')

In [103]:
# open database connection
connection = sqlite3.connect('data/l1000.db')
cursor = connection.cursor()

## Create cells table

In [109]:
cell_df = pandas.read_table('data/cellinfo/cellinfo.tsv.gz', compression='gzip', na_values=['-666', ''])
cell_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/cellinfo/cellinfo.tsv.gz'

## Create perts table

In [104]:
# Create table of perturbations
command = '''
CREATE TABLE perts
(
    pert_id TEXT PRIMARY KEY NOT NULL,
    pert_iname TEXT,
    pert_type TEXT NOT NULL,
    num_gold INTEGER NOT NULL,
    num_inst INTEGER NOT NULL,
    num_sig INTEGER NOT NULL,
    in_summly INTEGER NOT NULL,
    inchi_string TEXT,
    inchi_key TEXT,
    pubchem_cid INTEGER
);
'''
cursor.execute(command)

perts_columns = [col[1] for col in cursor.execute('PRAGMA table_info(perts);')]
perts_columns

['pert_id',
 'pert_iname',
 'pert_type',
 'num_gold',
 'num_inst',
 'num_sig',
 'in_summly',
 'inchi_string',
 'inchi_key',
 'pubchem_cid']

In [105]:
# read L1000 perturbagens
pert_df = pandas.read_table('data/pertinfo/pertinfo.tsv.gz', compression='gzip', na_values=['-666', ''])
pert_df = pert_df[perts_columns]
pert_df['pubchem_cid'] = pert_df.pubchem_cid.astype(int, raise_on_error=False)
pert_df['in_summly'] = pert_df.in_summly.map(lambda x: int(x is True))
pert_df.head()

,pert_id,pert_iname,pert_type,num_gold,num_inst,num_sig,in_summly,inchi_string,inchi_key,pubchem_cid
0,CSS001-ATTGCAT,ATTGCAT,trt_sh.css,0,0,7,0,NaN,NaN,NaN
1,CSS001-GAGGATA,GAGGATA,trt_sh.css,0,0,1,0,NaN,NaN,NaN
2,CSS001-TCAATGA,TCAATGA,trt_sh.css,0,0,7,0,NaN,NaN,NaN
3,CSS001-TCAGTTC,TCAGTTC,trt_sh.css,0,0,7,0,NaN,NaN,NaN
4,CSS001-TCCATCA,TCCATCA,trt_sh.css,0,0,1,0,NaN,NaN,NaN


In [107]:
# Create table of perturbations
pert_df.to_sql('perts', connection, if_exists='append', index=False)

## Create sigs tables

In [ ]:
# Create table of perturbations
command = '''
CREATE TABLE sigs
(
    sig_id TEXT PRIMARY KEY NOT NULL,
    pert_id TEXT NOT NULL,
    pert_itime TEXT NOT NULL,
    pert_idose TEXT NOT NULL,
    cell_id INTEGER NOT NULL,
    is_gold INTEGER NOT NULL,
    ngenes_modulated_dn_lm INTEGER NOT NULL,
    ngenes_modulated_dn_up INTEGER NOT NULL,
    FOREIGN KEY(pert_id) REFERENCES perts(pert_id),
    FOREIGN KEY(cell_id) REFERENCES cells(cell_id)
);
'''
cursor.execute(command)

sig_columns = [col[1] for col in cursor.execute('PRAGMA table_info(sigs);')]
sig_columns

In [108]:
# read L1000 perturbagens
pert_df = pandas.read_table('data/siginfo/siginfo.tsv.gz', compression='gzip', na_values=['-666', ''])
pert_df.head()

,sig_id,pert_id,pert_itime,distil_nsample,pert_idose,cell_id,pert_type,is_gold,distil_ss,ngenes_modulated_dn_lm,ngenes_modulated_up_lm
0,CVD001_HUH7_24H:BRD-A13020530-001-01-7:10,BRD-A13020530,24 h,1,10 µM,HUH7,trt_cp,False,13.82390,311,260
1,CVD001_HUH7_24H:BRD-K07762753-001-03-6:50,BRD-K07762753,24 h,2,50 µM,HUH7,trt_cp,True,14.55140,298,302
2,CPC004_PC3_6H:BRD-K34820100-001-02-1:10,BRD-K34820100,6 h,5,10 µM,PC3,trt_cp,False,4.15488,41,39
3,CPC004_PC3_6H:BRD-A22844106-001-16-1:10,BRD-A22844106,6 h,5,10 µM,PC3,trt_cp,False,2.93299,14,12
4,CPC004_PC3_6H:BRD-A55393291-001-05-7:10,BRD-A55393291,6 h,5,10 µM,PC3,trt_cp,False,1.96571,8,6


In [ ]:
# Create table of compound similarity
command = '''
CREATE TABLE similarities
(
    pert_0 TEXT NOT NULL,
    pert_1 TEXT NOT NULL,
    chemical REAL,
    FOREIGN KEY(pert_0) REFERENCES perts(pert_id),
    FOREIGN KEY(pert_1) REFERENCES perts(pert_id),
    PRIMARY KEY (pert_0, pert_1)
);
'''
cursor.execute(command);

In [72]:
#connection.commit()
connection.close()
os.unlink('data/l1000.db')